In [ ]:
import os
import numpy as np
import pandas as pd
import json

In [ ]:
results_path = os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models")
experiments_name = [folder for folder in os.listdir(results_path) 
                    if os.path.isdir(os.path.join(results_path, folder)) and os.path.exists(os.path.join(results_path, folder, 'metrics.json'))]

experiments_dict = {
    'datetime':[],
    'dataset':[],
    'model':[],
    'distribution_head':[],
    'scaling':[],
    'model_params':[],
    'epochs':[],
    'early_stop':[],
    'actual':[],
    'forecast':[],
    'metrics':[]
}

baselines = ['iETS', 'ZeroForecast','EmpQ', 'ML_negbin', 'ML_zero-inf-pois', 'ML_tweedie', 'ML_poisson', 'ML_tweedie-fix']

for exp in experiments_name:
    skip = False
    for b in baselines:
        if b in exp: skip = True
    exp_split = exp.split('__')
    experiments_dict['dataset'].append( exp_split[1] )
    experiments_dict['model'].append( exp_split[0] )
    experiments_dict['actual'].append( os.path.join(results_path,exp,'actuals.npy') )
    experiments_dict['forecast'].append( os.path.join(results_path,exp,'forecasts.npy') )
    experiments_dict['metrics'].append( json.load(open(os.path.join(results_path,exp,'metrics.json'), "r")) )
    if skip:
        experiments_dict['distribution_head'].append( "-" )
        experiments_dict['scaling'].append( '-' )
        experiments_dict['epochs'].append( np.nan )
        experiments_dict['early_stop'].append( np.nan )
        experiments_dict['model_params'].append( np.nan )
        experiments_dict['datetime'].append( exp_split[2] )
        continue
    exp_info = json.load(open(os.path.join(results_path,exp,'experiment.json'), "r"))
    experiments_dict['distribution_head'].append( exp_info['distribution_head'] )
    experiments_dict['scaling'].append( exp_info['scaling'] if exp_info['scaling'] else '-' )
    experiments_dict['epochs'].append( exp_info['epoch'] )
    experiments_dict['early_stop'].append( exp_info['early_stop'] )
    experiments_dict['model_params'].append( json.load(open(os.path.join(results_path,exp,'model_params.json'), "r")) )
    experiments_dict['datetime'].append( exp_split[4] )

In [ ]:
experiments_df = pd.DataFrame(experiments_dict)
experiments_df['datetime'] = pd.to_datetime(experiments_df['datetime'], format="%Y-%m-%d-%H-%M-%S-%f")
# -- 
experiments_df = experiments_df[(experiments_df.dataset == "M5") & ((experiments_df.model.isin(["deepAR"])) | experiments_df.model.isin(baselines)) &
                                (~experiments_df.distribution_head.isin(['tweedie-priors']) )]
experiments_df.sort_values('datetime', inplace=True)
# --
experiments_df.set_index(['datetime','dataset','model','distribution_head','scaling'], inplace=True)

quantile_losses = pd.DataFrame([values['metrics']['rho_risk_nan']['intermittent_and_lumpy'] for _, values in experiments_df.iterrows()]).round(3).astype(float)
quantile_losses.set_index(experiments_df.index, inplace=True)
quantile_losses.sort_values(['dataset', 'scaling', 'distribution_head', 'datetime'], inplace=True)
styled_quantile_losses = quantile_losses.iloc[:,2:].style.apply(lambda x: (x <= x.groupby('scaling').transform('min')).map({
        True: 'background-color: yellow',
        False: None,
    })).apply(lambda col: ['background-color: green' if v == col.min() else '' for v in col])

styled_quantile_losses

In [ ]:
from dataloader import load_raw
from measures import compute_intermittent_indicators

ts_classification = {}

In [ ]:
dataset = 'carparts'
model = 'deepAR'
scaling = 'mase'
distribution_heads = ['negbin','poisson', 'tweedie', 'tweedie-fix', 'zero-inf-pois']
subset = 'intermittent'
quantiles = [0.25, 0.5, 0.8, 0.9, 0.95, 0.99]

if dataset not in ts_classification.keys():
    data_raw, _ = load_raw(dataset, os.path.join(  "../data"))
    adi, cv2 = compute_intermittent_indicators(data_raw)
    ts_classification[dataset] = {'intermittent' : np.logical_and(adi >= 1.32, cv2 < .49),
                                  'intermittent_and_lumpy' : adi >= 1.32}


#ranks(dataset, model, scaling, distribution_heads, subset)
    

scores_to_rank = {'q'+str(q) : {} for q in quantiles}
rank = {}

for experiments_name in os.listdir(results_path):
    exp = experiments_name.split('__')
    if exp[0] == model and exp[1] == dataset and exp[3] == scaling and exp[2] in distribution_heads:
        forecasts = np.load(os.path.join(results_path, experiments_name, 'forecasts.npy'))
        qforecasts = np.transpose(np.quantile(forecasts, quantiles, axis=1), (1,2,0))
        actuals = np.load(os.path.join(results_path, experiments_name, 'actuals.npy'))
        
        for j in range(len(quantiles)):
            scores_to_rank['q'+str(quantiles[j])][exp[2]] = 2 * np.mean(np.abs((qforecasts[:,:,j] - actuals) * ((actuals <= qforecasts[:,:,j]) - quantiles[j])), axis=1)
        
for q in quantiles:
    quantile_to_rank = pd.DataFrame(scores_to_rank['q'+str(q)]).iloc[ts_classification[dataset][subset]]
    quantile_to_rank = quantile_to_rank[sorted(quantile_to_rank.columns)]

    rank['q'+str(q)] = np.mean(1 + np.argsort(np.argsort(quantile_to_rank.values, axis=1), axis = 1), axis=0)


pd.DataFrame(rank, index = sorted(quantile_to_rank.columns)).style.background_gradient(low=2, high=3, axis=0)


# LaTeX formatting #


In [ ]:
replace_map = {'deepAR':'DeepAR', 'iETS':'iETS MNN', 'ZeroForecast':'Dirac', 'transformer':'Transformer', 'EmpQ':'Empirical quantiles',
'poisson':'Poisson', 'negbin':'Negative Binomial', 'tweedie':'Tweedie', 'tweedie-fix':'Tweedie (fixed dispersion)', 'zero-inf-pois':'Zero-inflated Poisson', '-':'/',
'-':'/', 'mean':'Mean', 'mean-demand':'Mean demand', 'mase':'MASE'}

experiments_df = pd.DataFrame(experiments_dict)
experiments_df.set_index(['datetime','dataset','model','distribution_head','scaling'], inplace=True)

df_intermittent = pd.DataFrame([values['metrics']['quantile_loss']['intermittent'] for _, values in experiments_df.iterrows()]).round(3).astype(float)
df_intermittent_and_lumpy = pd.DataFrame([values['metrics']['quantile_loss']['intermittent_and_lumpy'] for _, values in experiments_df.iterrows()]).round(3).astype(float)

ql = pd.concat((df_intermittent, df_intermittent_and_lumpy), axis=1, keys=['Intermittent', 'Intermittent and lumpy'])
ql.set_index(experiments_df.index, inplace=True)
ql.rename_axis(['Datetime', 'Dataset', 'Model','Likelihood', 'Scaling'], inplace=True)
ql.reset_index(level = ('Model', 'Likelihood', 'Scaling'), inplace=True)
ql.replace(replace_map, inplace=True)
ql.set_index(['Model', 'Likelihood', 'Scaling'], append=True, inplace=True)
ql.sort_values(['Dataset', 'Scaling', 'Likelihood', 'Datetime'], inplace=True)
ql

In [ ]:
def print_latex(ql_sliced):
    latex = ql_sliced.style.apply(lambda x: (x <= x.groupby('Scaling').transform('min')).map({
        True: 'cellcolor:{yellow}',
        False: None,
    })).apply(lambda col: ['cellcolor:{green}' if v == col.min() else '' for v in col]).to_latex()
    latex_list = latex.splitlines()
    latex_list[0] =  r'\tiny ' + latex_list[0].replace('lll', 'lll|')
    latex_list[1] = latex_list[1].replace(' & '.join(latex_list[1].split(' & ')[:3]), ' & '.join(latex_list[2].split(' & ')[:3]))
    latex_list[2] = r'\toprule'
    latex_list = [r'\begin{center}'] + latex_list + [r'\end{center}']
    latex_list = ['\cmidrule{4-9}\n' + line if line.startswith(' & Negative Binomial') else line for line in latex_list]      

    latex = '\n'.join(latex_list)
    latex = latex.replace('000', '')
    print(latex)


ql_sliced = ql['Intermittent'].loc[(ql.index.get_level_values('Dataset') == 'RAF') & (ql.index.get_level_values('Model').isin(['DeepAR']))].droplevel(['Datetime', 'Dataset'])
print_latex(ql_sliced)


In [ ]:
models = {"Baselines":["iETS MNN", "Empirical quantiles", 'Dirac'], "Transformer":["Transformer"], "DeepAR":["DeepAR"]}


for dataset in [ 'M5']:

    for subset in ['Intermittent', 'Intermittent and lumpy']:

        print("\n" + r"\newpage" + "\n" + r"\section{\texttt{" + dataset + "}, " + subset.lower() + " time series}")
        
        for k in models.keys():

            print("\n\subsection{" + k + "}")

            ql_sliced = ql[subset].loc[(ql.index.get_level_values('Dataset') == dataset) & (ql.index.get_level_values('Model').isin(models[k]))].droplevel(['Datetime', 'Dataset'])

            print_latex(ql_sliced)


        

Use this chunk of code when you want to add a metrics to past models.


In [ ]:
from tqdm import tqdm
import os
import json
import numpy as np
from dataloader import load_raw
from measures import rho_risk_sample, rho_risk, compute_intermittent_indicators


ds_dict = {}


for folder in tqdm(os.listdir( os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models"))):


    if os.path.isfile(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'forecasts.npy')):

        exp = json.load(open(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'experiment.json')))
        dataset = exp['dataset']
    
        if dataset not in ds_dict.keys():
            data_raw, data_infos = load_raw(dataset, '../data/')
            data_raw = data_raw.dropna()
        
            adi, cv2 = compute_intermittent_indicators(data_raw)
            idx_intermittent = np.logical_and(adi >= 1.32, cv2 < .49)
            idx_intermittent_and_lumpy = adi >= 1.32
        
            ds_dict[dataset] = {'idx_intermittent':idx_intermittent, 'idx_intermittent_and_lumpy':idx_intermittent_and_lumpy}
    
        idx_intermittent = ds_dict[dataset]['idx_intermittent']
        idx_intermittent_and_lumpy = ds_dict[dataset]['idx_intermittent_and_lumpy']
        
        forecasts = np.load(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'forecasts.npy'))
        actuals = np.load(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'actuals.npy'))

        assert forecasts.shape[0] == actuals.shape[0]
        assert forecasts.shape[2] == actuals.shape[1]

        metrics = json.load(open(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'metrics.json')))

        metrics['rho_risk_nan'] = {'all' : rho_risk_sample(actuals, forecasts, zero_denom = np.nan),
                                   'intermittent' : rho_risk_sample(actuals[idx_intermittent,:], forecasts[idx_intermittent,:,:], zero_denom = np.nan),
                                   'intermittent_and_lumpy' : rho_risk_sample(actuals[idx_intermittent_and_lumpy,:], forecasts[idx_intermittent_and_lumpy,:,:], zero_denom = np.nan)}
        metrics['rho_risk_1'] = {'all' : rho_risk_sample(actuals, forecasts, zero_denom = 1.),
                                 'intermittent' : rho_risk_sample(actuals[idx_intermittent,:], forecasts[idx_intermittent,:,:], zero_denom = 1.),
                                 'intermittent_and_lumpy' : rho_risk_sample(actuals[idx_intermittent_and_lumpy,:], forecasts[idx_intermittent_and_lumpy,:,:], zero_denom = 1.)}
        json.dump(metrics, open(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'metrics.json'), "w"))
        
    elif os.path.isfile(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'qforecasts.npy')):

        dataset = folder.split('__')[1]
    
        if dataset not in ds_dict.keys():
            data_raw, data_infos = load_raw(dataset, '../data/')
            data_raw = data_raw.dropna()
        
            adi, cv2 = compute_intermittent_indicators(data_raw)
            idx_intermittent = np.logical_and(adi >= 1.32, cv2 < .49)
            idx_intermittent_and_lumpy = adi >= 1.32
        
            ds_dict[dataset] = {'idx_intermittent':idx_intermittent, 'idx_intermittent_and_lumpy':idx_intermittent_and_lumpy}

        idx_intermittent = ds_dict[dataset]['idx_intermittent']
        idx_intermittent_and_lumpy = ds_dict[dataset]['idx_intermittent_and_lumpy']

        qforecasts = np.load(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'qforecasts.npy'))
        actuals = np.load(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'actuals.npy'))

        assert qforecasts.shape[0] == actuals.shape[0]
        assert qforecasts.shape[1] == actuals.shape[1]

        metrics = json.load(open(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'metrics.json')))
                          
        metrics['rho_risk_nan'] = {'all' : rho_risk(actuals, qforecasts, zero_denom = np.nan),
                                   'intermittent' : rho_risk(actuals[idx_intermittent,:], qforecasts[idx_intermittent,:,:], zero_denom = np.nan),
                                   'intermittent_and_lumpy' : rho_risk(actuals[idx_intermittent_and_lumpy,:], qforecasts[idx_intermittent_and_lumpy,:,:], zero_denom = np.nan)}
        metrics['rho_risk_1'] = {'all' : rho_risk(actuals, qforecasts, zero_denom = 1.),
                                 'intermittent' : rho_risk(actuals[idx_intermittent,:], qforecasts[idx_intermittent,:,:], zero_denom = 1.),
                                 'intermittent_and_lumpy' : rho_risk(actuals[idx_intermittent_and_lumpy,:], qforecasts[idx_intermittent_and_lumpy,:,:], zero_denom = 1.)}

        json.dump(metrics, open(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models", folder, 'metrics.json'), "w"))
        
    elif folder == '.DS_Store':
        continue
    
    else:
        print('AIUTOOOOOOOO', folder)

Use this code if you want to check if there are missing models

In [ ]:
import itertools

trained_folders = os.listdir(os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models"))

models = ['deepAR', 'transformer']
datasets = ['M5', 'OnlineRetail', 'carparts', 'RAF', 'Auto']
likelihoods = ['negbin', 'poisson', 'tweedie', 'tweedie-fix', 'zero-inf-pois']
scalings = ['none', 'mase', 'mean', 'mean-demand']

for el in itertools.product(models, datasets, likelihoods, scalings):
    code = "__".join(el)
    #if not any(folder.startswith(code) for folder in trained_folders):
        #print(code)
    count_folder = sum([1 if folder.startswith(code+'__') else 0 for folder in trained_folders])
    if count_folder != 1:
        print(code, count_folder)


In [ ]:
for dataset in ['M5', 'OnlineRetail', 'carparts', 'RAF', 'Auto']:
    data_raw, data_infos = load_raw(dataset, '../data/')
    data_raw = data_raw.dropna()
    print(dataset, '\t',np.sum(np.all(np.array(data_raw.iloc[:,-data_infos['h']:]) == 0, axis=1)), len(data_raw), '\t',
          str(100*np.sum(np.array(data_raw.iloc[:,-data_infos['h']:]) == 0)/(len(data_raw)*data_infos['h']))+'%')